<span style="color: purple">

Load in stored variables:

</span>

In [1]:
%store -r data_dir sites_gdf siskiyou_forest_gdf padres_forest_gdf

<span style="color: purple">

Import packages:

</span>

In [12]:
# Import necessary packages
import os

import matplotlib.pyplot as plt # Overlay pandas and xarray plots
import pandas as pd # Aggregating and data manipulation
from tqdm.notebook import tqdm # Progress bars on loops
import xarray as xr

## STEP 2c: DATA ACCESS - CLIMATE MODELS

You can use MACAv2 data for historical and future climate data. Be sure
to compare at least two 30-year time periods (e.g. historical vs. 10
years in the future) for at least four of the CMIP models. Overall, you
should be downloading at least 8 climate rasters for each of your sites,
for a total of 16. **You will *need* to use loops and/or functions to do
this cleanly!**.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download MACAv2 data for a particular climate model, emissions
scenario, spatial domain, and time frame. Then, use loops to download
and organize the 16+ rasters you will need to complete this section. The
<a
href="http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_macav2_catalog2.html">MACAv2
dataset is accessible from their Thredds server</a>. Include an
arrangement of sites, models, emissions scenarios, and time periods that
will help you to answer your scientific question.</p></div></div>

<span style="color: purple">

List of climate models:

</span>

<span style="color: magenta">

* Warm and wet: CanESM2

</span>

<span style="color: limegreen">

* Warm and dry: MIROC-ESM-CHEM

</span>

<span style="color: teal">

* Cold and wet: MRI-CGCM3

</span>

<span style="color: black">

* Cold and dry: GFDL-ESM2M

</span>

<span style="color: purple">

<span style="color: purple">

The time periods I will compare are:

* 2035 to 2065 - this will allow for a summary of climate for 2050
* 2065 to 2095 - this will allow for a summary of climate for 2080

</span>

In [3]:
# Make MACA data directory
maca_dir = os.path.join(data_dir, 'maca-dir')
os.makedirs(maca_dir, exist_ok=True)
maca_dir

maca_pattern = os.path.join(maca_dir, '*.nc')
maca_pattern

'C:\\Users\\riede\\earth-analytics\\data\\spring-2025-habitat-suitability\\maca-dir\\*.nc'

<span style="color: purple">

Define functions:

</span>

In [4]:
# Define function to convert longitude/latitude
def convert_longitude(longitude):
    """
    Convert longitude range from 0-360 to -180-180
    
    Parameters
    ----------
    longitude : float
        `longitude` values of dataset
    
    Returns
    -------
    longitude : float
        Longitude values converted from 0-360 to -180-180"""
    return (longitude - 360) if longitude > 180 else longitude

In [36]:
def process_maca_da(site_dict,
                    years_list,
                    models_list,
                    rcp_value,
                    maca_data_dir):
    """
    Create a URL to access MACA climate precipitation data from the Northwest Knowledge Network.
    Use the URL to download, open, squeeze, and crop raster climate precipitation data.
    
    Parameters
    ----------
    site_dict : dict
        Dictionary of site GeoDataFrames
    years_list : list
        List of 5 year time periods
    models_list : list
        List of climate model names
    rcp_value : str
        Representative Concentration Pathway value
    maca_data_dir: str
        file path where maca NetCDF files will be stored

    Returns
    -------
    maca_da_list : list
        A list of DataArrays of climate data (precipitation) for
        the different rasters covering the area of interest
    """
    # empty list to store processed MACA DataArrays
    maca_da_list = []

    for site_name, site_gdf in tqdm(site_dict.items()):
        for date_range in tqdm(years_list):
            for model in tqdm(models_list):
                # Define MACA url
                maca_url = (
                    'http://thredds.northwestknowledge.net:8080/thredds/dodsC'
                    '/MACAV2'
                    f'/{model}'
                    '/macav2metdata_pr'
                    f'_{model}_r1i1p1'
                    f'_{rcp_value}'
                    f'_{date_range}_CONUS'
                    '_monthly.nc')
                # check maca_url
                print(maca_url)

                # Only download MACA data once
                maca_path = os.path.join(maca_data_dir, f'maca_{model}_{site_name}_{rcp_value}_{date_range}_CONUS_monthly.nc')
                print(maca_path)
                if not os.path.exists(maca_path):
                    maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                    maca_da.to_netcdf(maca_path)

                # Open & Squeeze MACA dataset
                maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                print('Opened & Squeezed')

                # Define bounds
                bounds_maca = (site_gdf
                                .to_crs(maca_da.rio.crs)
                                .total_bounds)

                # Change maca_ds longitude values to match the foest gdfs
                maca_da = maca_da.assign_coords(
                    lon=("lon", [convert_longitude(l) for l in maca_da.lon.values]))

                # Set spatial dimensions of maca_da
                maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

                # Crop maca_da
                maca_da = maca_da.rio.clip_box(*bounds_maca)
                print('Cropped')

                # Add cropped da to a dictionary w/ metadata and then the maca_da_list
                maca_da_list.append(dict(
                    site_name=site_name,
                    climate_model=model,
                    date_range=date_range,
                    da=maca_da))
    return maca_da_list

<span style="color: purple">

Set up variables to use in process_maca_da function:

</span>

In [ ]:
# site dictionary
siskiyou_padres_dict = {
    'SiskiyouForest': siskiyou_forest_gdf,
    'LosPadresForest': padres_forest_gdf
}

# list of time periods to create MACA urls
years_2050 = ['2036_2040', '2041_2045', '2046_2050',
         '2051_2055', '2056_2060', '2061_2065']

years_2080 = ['2066_2070', '2071_2075', '2076_2080',
              '2081_2085', '2086_2090', '2091_2095']

# list of climate models
climate_models = ['CanESM2', 'MIROC-ESM-CHEM',
          'MRI-CGCM3', 'GFDL-ESM2M']

<span style="color: purple">

Process MACA DataArrays for 2050 time period for both sites for all 4 climate models:

</span>

In [37]:
maca_pr_2050_da_list = process_maca_da(siskiyou_padres_dict,
                                       years_2050,
                                       climate_models,
                                       'rcp85',
                                       maca_dir)

maca_pr_2050_da_list

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFDL-E

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

[{'site_name': 'SiskiyouForest',
  'climate_model': 'CanESM2',
  'date_range': '2036_2040',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 values with dtype=float32]
  Coordinates:
    * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
    * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
    * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
      crs      int64 8B 0
  Attributes:
      long_name:      Monthly Precipitation Amount
      units:          mm
      standard_name:  precipitation
      cell_methods:   time: sum(interval: 24 hours): sum over days
      comments:       Total monthly precipitation at surface: includes both liq...
      _ChunkSizes:    [ 10  44 107]},
 {'site_name': 'SiskiyouForest',
  'climate_model': 'MIROC-ESM-CHEM',
  'date_range': '2036_2040',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 value

In [39]:
maca_pr_2050_df = pd.DataFrame(maca_pr_2050_da_list)

# Display maca_df w/o DataArray column
display(maca_pr_2050_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_pr_2050_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2036_2040
1,SiskiyouForest,MIROC-ESM-CHEM,2036_2040
2,SiskiyouForest,MRI-CGCM3,2036_2040
3,SiskiyouForest,GFDL-ESM2M,2036_2040
4,SiskiyouForest,CanESM2,2041_2045
5,SiskiyouForest,MIROC-ESM-CHEM,2041_2045
6,SiskiyouForest,MRI-CGCM3,2041_2045
7,SiskiyouForest,GFDL-ESM2M,2041_2045
8,SiskiyouForest,CanESM2,2046_2050
9,SiskiyouForest,MIROC-ESM-CHEM,2046_2050


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

<span style="color: purple">

Process MACA DataArrays for 2080 time period for both sites for all 4 climate models:

</span>

In [40]:
maca_pr_2080_da_list = process_maca_da(siskiyou_padres_dict,
                                       years_2080,
                                       climate_models,
                                       'rcp85',
                                       maca_dir)

maca_pr_2080_da_list

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2066_2070_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_SiskiyouForest_rcp85_2066_2070_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2066_2070_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2066_2070_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2066_2070_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_SiskiyouForest_rcp85_2066_2070_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFDL-E

  0%|          | 0/4 [00:00<?, ?it/s]

...

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2086_2090_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2086_2090_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2086_2090_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2086_2090_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2086_2090_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2086_2090_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2091_2095_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2091_2095_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2091_2095_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2091_2095_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

[{'site_name': 'SiskiyouForest',
  'climate_model': 'CanESM2',
  'date_range': '2066_2070',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 values with dtype=float32]
  Coordinates:
    * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
    * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
    * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
      crs      int64 8B 0
  Attributes:
      long_name:      Monthly Precipitation Amount
      units:          mm
      standard_name:  precipitation
      cell_methods:   time: sum(interval: 24 hours): sum over days
      comments:       Total monthly precipitation at surface: includes both liq...
      _ChunkSizes:    [ 10  44 107]},
 {'site_name': 'SiskiyouForest',
  'climate_model': 'MIROC-ESM-CHEM',
  'date_range': '2066_2070',
  'da': <xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
  [40500 value

In [41]:
maca_pr_2080_df = pd.DataFrame(maca_pr_2080_da_list)

# Display maca_df w/o DataArray column
display(maca_pr_2080_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_pr_2080_df.da.values[0]

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2066_2070
1,SiskiyouForest,MIROC-ESM-CHEM,2066_2070
2,SiskiyouForest,MRI-CGCM3,2066_2070
3,SiskiyouForest,GFDL-ESM2M,2066_2070
4,SiskiyouForest,CanESM2,2071_2075
5,SiskiyouForest,MIROC-ESM-CHEM,2071_2075
6,SiskiyouForest,MRI-CGCM3,2071_2075
7,SiskiyouForest,GFDL-ESM2M,2071_2075
8,SiskiyouForest,CanESM2,2076_2080
9,SiskiyouForest,MIROC-ESM-CHEM,2076_2080


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2066-01-15 00:00:00 ... 2070-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

# **BREAK**

In [26]:
# empty list to store MACA DataArrays
maca_da_list = []

# list of time periods to create MACA urls
years_2050 = ['2036_2040', '2041_2045', '2046_2050',
         '2051_2055', '2056_2060', '2061_2065']

years_2080 = ['2066_2070', '2071_2075', '2076_2080',
              '2081_2085', '2086_2090', '2091_2095']

# list of climate models
climate_models = ['CanESM2', 'MIROC-ESM-CHEM',
          'MRI-CGCM3', 'GFDL-ESM2M']

for site_name, site_gdf in tqdm({
    'SiskiyouForest': siskiyou_forest_gdf,
    'LosPadresForest': padres_forest_gdf}.items()):
    for date_range in tqdm(years_2050):
        for model in tqdm(climate_models):
            # Define MACA url
            maca_url = (
                'http://thredds.northwestknowledge.net:8080/thredds/dodsC'
                '/MACAV2'
                f'/{model}'
                '/macav2metdata_pr'
                f'_{model}_r1i1p1'
                '_rcp85'
                f'_{date_range}_CONUS'
                '_monthly.nc')
            # check maca_url
            print(maca_url)

            # Only download MACA data once
            maca_path = os.path.join(maca_dir, f'maca_{model}_{site_name}_rcp85_{date_range}_CONUS_monthly.nc')
            print(maca_path)
            if not os.path.exists(maca_path):
                 maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                 maca_da.to_netcdf(maca_path)

            # Open & Squeeze MACA dataset
            maca_da = xr.open_dataset(maca_url).squeeze().precipitation
            print('Opened & Squeezed')

            # Define bounds
            bounds_maca = (site_gdf
                            .to_crs(maca_da.rio.crs)
                            .total_bounds)

            # Change maca_ds longitude values to match the grasslands gdfs
            maca_da = maca_da.assign_coords(
                lon=("lon", [convert_longitude(l) for l in maca_da.lon.values]))

            # Set spatial dimensions of maca_da
            maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

            # Crop maca_da
            maca_da = maca_da.rio.clip_box(*bounds_maca)
            print('Cropped')

            # Add cropped da to a dictionary w/ metadata and then the maca_da_list
            maca_da_list.append(dict(
                site_name=site_name,
                climate_model=model,
                date_range=date_range,
                da=maca_da))
            

maca_df = pd.DataFrame(maca_da_list)

# Display maca_df w/o DataArray column
display(maca_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_df.da.values[0]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFDL-E

  0%|          | 0/4 [00:00<?, ?it/s]

...

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2036_2040
1,SiskiyouForest,MIROC-ESM-CHEM,2036_2040
2,SiskiyouForest,MRI-CGCM3,2036_2040
3,SiskiyouForest,GFDL-ESM2M,2036_2040
4,SiskiyouForest,CanESM2,2041_2045
5,SiskiyouForest,MIROC-ESM-CHEM,2041_2045
6,SiskiyouForest,MRI-CGCM3,2041_2045
7,SiskiyouForest,GFDL-ESM2M,2041_2045
8,SiskiyouForest,CanESM2,2046_2050
9,SiskiyouForest,MIROC-ESM-CHEM,2046_2050


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [ ]:
# testing part of the for loop
maca_test_path = os.path.join(maca_dir, 'maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc')

if not os.path.exists(maca_test_path):
                 maca_test_da = xr.open_dataset('http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc').squeeze().precipitation
                 maca_da.to_netcdf(maca_test_path)

maca_test_da = xr.open_dataset('http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc').squeeze().precipitation

In [24]:
maca_test_da

<xarray.DataArray 'precipitation' (time: 60, lat: 585, lon: 1386)> Size: 195MB
[48648600 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 5kB 25.06 25.1 25.15 25.19 ... 49.27 49.31 49.35 49.4
    crs      int32 4B 1
  * lon      (lon) float64 11kB 235.2 235.3 235.3 235.4 ... 292.9 292.9 292.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    grid_mapping:   crs
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

example CanESM2 link:

http://thredds.northwestknowledge.net:8080/thredds/catalog/MACAV2/CanESM2/catalog.html?dataset=REACCHDatasetScan_CanESM2_MACAV2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc

example miroc-esm-chem link:
http://thredds.northwestknowledge.net:8080/thredds/catalog/MACAV2/MIROC-ESM-CHEM/catalog.html?dataset=REACCHDatasetScan_MIROC-ESM-CHEM_MACAV2/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc

Example MRI-CGCM3 link, but from the [page](http://thredds.northwestknowledge.net:8080/thredds/catalog/MACAV2/MRI-CGCM3/catalog.html?dataset=REACCHDatasetScan_MRI-CGCM3_MACAV2/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc) for the specific link, not the list of all the links:
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc.html

**fxn attempt of maca download:**

In [ ]:
def process_maca_da(site_dict,
                    years_list,
                    models_list,
                    rcp_value,
                    maca_data_dir):
    """
    Create a URL to access MACA climate precipitation data from the Northwest Knowledge Network.
    Use the URL to download, open, squeeze, and crop raster climate precipitation data.
    
    Parameters
    ----------
    site_dict : dict
        Dictionary of site GeoDataFrames
    years_list : list
        List of 5 year time periods
    models_list : list
        List of climate model names
    rcp_value : str
        Representative Concentration Pathway value
    maca_data_dir: str
        file path where maca NetCDF files will be stored

    Returns
    -------
    maca_da_list : list
        A list of DataArrays of climate data (precipitation) for
        the different rasters covering the area of interest
    """
    # empty list to store processed MACA DataArrays
    maca_da_list = []

    for site_name, site_gdf in tqdm(site_dict.items()):
        for date_range in tqdm(years_list):
            for model in tqdm(models_list):
                # Define MACA url
                maca_url = (
                    'http://thredds.northwestknowledge.net:8080/thredds/dodsC'
                    '/MACAV2'
                    f'/{model}'
                    '/macav2metdata_pr'
                    f'_{model}_r1i1p1'
                    f'_{rcp_value}'
                    f'_{date_range}_CONUS'
                    '_monthly.nc')
                # check maca_url
                print(maca_url)

                # Only download MACA data once
                maca_path = os.path.join(maca_data_dir, f'maca_{model}_{site_name}_{rcp_value}_{date_range}_CONUS_monthly.nc')
                print(maca_path)
                if not os.path.exists(maca_path):
                    maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                    maca_da.to_netcdf(maca_path)

                # Open & Squeeze MACA dataset
                maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                print('Opened & Squeezed')

                # Define bounds
                bounds_maca = (site_gdf
                                .to_crs(maca_da.rio.crs)
                                .total_bounds)

                # Change maca_ds longitude values to match the foest gdfs
                maca_da = maca_da.assign_coords(
                    lon=("lon", [convert_longitude(l) for l in maca_da.lon.values]))

                # Set spatial dimensions of maca_da
                maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')

                # Crop maca_da
                maca_da = maca_da.rio.clip_box(*bounds_maca)
                print('Cropped')

                # Add cropped da to a dictionary w/ metadata and then the maca_da_list
                maca_da_list.append(dict(
                    site_name=site_name,
                    climate_model=model,
                    date_range=date_range,
                    da=maca_da))
    return maca_da_list

In [35]:
# list of climate models
climate_models = ['CanESM2', 'MIROC-ESM-CHEM',
          'MRI-CGCM3', 'GFDL-ESM2M']

siskiyou_padres_dict = {
    'SiskiyouForest': siskiyou_forest_gdf,
    'LosPadresForest': padres_forest_gdf
}

process_maca_da(siskiyou_padres_dict,
                                   years_2050,
                                   climate_models,
                                   'rcp85',
                                   maca_dir)

maca_pr_2050_df = pd.DataFrame(maca_da_list)

# Display maca_df w/o DataArray column
display(maca_pr_2050_df.drop('da', axis='columns'))

# Pull out the data array for the first row
maca_df.da.values[0]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2036_2040_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_SiskiyouForest_rcp85_2036_2040_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFDL-E

  0%|          | 0/4 [00:00<?, ?it/s]

...

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2056_2060_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2056_2060_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

  0%|          | 0/4 [00:00<?, ?it/s]

http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/CanESM2/macav2metdata_pr_CanESM2_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_CanESM2_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MIROC-ESM-CHEM/macav2metdata_pr_MIROC-ESM-CHEM_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MIROC-ESM-CHEM_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/MRI-CGCM3/macav2metdata_pr_MRI-CGCM3_r1i1p1_rcp85_2061_2065_CONUS_monthly.nc
C:\Users\riede\earth-analytics\data\spring-2025-habitat-suitability\maca-dir\maca_MRI-CGCM3_LosPadresForest_rcp85_2061_2065_CONUS_monthly.nc
Opened & Squeezed
Cropped
http://thredds.northwestknowledge.net:8080/thredds/dodsC/MACAV2/GFD

,site_name,climate_model,date_range
0,SiskiyouForest,CanESM2,2036_2040
1,SiskiyouForest,MIROC-ESM-CHEM,2036_2040
2,SiskiyouForest,MRI-CGCM3,2036_2040
3,SiskiyouForest,GFDL-ESM2M,2036_2040
4,SiskiyouForest,CanESM2,2041_2045
5,SiskiyouForest,MIROC-ESM-CHEM,2041_2045
6,SiskiyouForest,MRI-CGCM3,2041_2045
7,SiskiyouForest,GFDL-ESM2M,2041_2045
8,SiskiyouForest,CanESM2,2046_2050
9,SiskiyouForest,MIROC-ESM-CHEM,2046_2050


<xarray.DataArray 'precipitation' (time: 60, lat: 25, lon: 27)> Size: 162kB
[40500 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 200B 41.9 41.94 41.98 42.02 ... 42.81 42.85 42.9
  * time     (time) object 480B 2036-01-15 00:00:00 ... 2040-12-15 00:00:00
  * lon      (lon) float64 216B -124.4 -124.4 -124.3 ... -123.4 -123.4 -123.3
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [ 10  44 107]

In [34]:
maca_pr_2050_das.head()

AttributeError: 'NoneType' object has no attribute 'head'

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Make sure to include a description of the climate data and how you
selected your models. Include a citation of the MACAv2 data</p></div></div>

YOUR CLIMATE DATA DESCRIPTION AND CITATIONS HERE